In [1]:
import numpy as np

In [2]:
def get_valid_float(a:str):
    flag=1
    while flag:
        try:
            out=float(input(a))
            flag=0
        except:
            print("Error: Enter a valid floating point value")
    return out

In [3]:
def create_plate(x: int, y: int):
    '''This function initializes the domain for computation based on the length and bredth of the geometry
        INPUT : Length,Bredth
        OUTPUT: 2D array
        Aux INPUT : Mesh Element dimensions
        '''
    try:
        while True:
            dx = get_valid_float ("Enter the element size on X-axis")
            dy = get_valid_float ("Enter the element size on Y-axis")
            if (x//dx)>3 and (y//dy)>3:
                domain = np.zeros((int(x//dx),int(y//dy)))
                break
            else:
                print("The elemant size is too large compared to plate size")
            
    except:
        print("Error: Given plate size is not valid")
        return 0
    
    return domain

In [4]:
def_boundary_temp=None

In [5]:
def boundary_conditions (domain: np.ndarray,left:float=def_boundary_temp, top:float=def_boundary_temp, right:float=def_boundary_temp, bottom:float=def_boundary_temp):
    '''Replaces the boundary values in the domain with the speccified values
        INPUT : Temperatures(t1,t2,t3,t4)
        |Takes Room Temperature if not specified|
        OUTPUT : Changes are made in-place at the array'''
    if type(domain)!=int:
        try:
            n1,n2 = domain.shape
            domain[:,0] = left
            domain[0] = top
            domain[:,n2-1] = right
            domain[n1-1] = bottom
            domain[0,0]=(left+top)/2
            domain[0,n2-1]=(top+right)/2
            domain[n1-1,n2-1]=(right+bottom)/2
            domain[n1-1,0]=(left+bottom)/2
        except:
            print("Given inputs are not valid")
            return 0
        return domain
    else:
        return 0

In [6]:
def libman_elip_pde_solver(domain : np.ndarray, lamb: float=1.5, error: float=1):
    if lamb>1 and lamb<2:
        err=0
        n1,n2=domain.shape
        if n1>2 and n2>2:
            while True:
                err=0
                for y in range(1,n1-1):
                    for x in range(1,n2-1):
                        temp=domain[x,y]
                        domain[x,y]=(domain[x-1,y]+domain[x,y-1]+domain[x+1,y]+domain[x,y+1])/4
                        domain[x,y]=(lamb*domain[x,y])+((1-lamb)*temp)
                        if abs(((domain[x,y]-temp)/domain[x,y])*100) >= err:
                            err = abs(((domain[x,y]-temp)/domain[x,y])*100)
                if err<error:
                    break
        else:
            return domain
            
    else:
        print("Error: lamb value should be between 1 to 2")
                    
    return domain,err

In [7]:
def analyse_plate(size_x: float,size_y: float,left:float,top:float,right:float,bottom:float, lamb:float=1.5,error_percentage:float=1):
    #set the default boundary temperature
    def_boundary_temp = get_valid_float("Enter the default boundary temp of the rectangular plate")
    # left- temperature of left side boundary of the plate
    # similaryly top, right , bottom follows correspondingly
    initalised_plate=boundary_conditions(create_plate(size_x,size_y),left,top,right,bottom)
    # An array initalized with boundary condition is passed on to pde solver
    steady_state_temp_dist=libman_elip_pde_solver(initalised_plate)
    return steady_state_temp_dist

In [8]:
# example 1 direct analysis with analyse plate function. 
# plate size 5 X 5 : first two parameters take plate x,y
# followed by left,top,right,bottom sides boundary condition. 
# followed by a control constant ranging between 1 to 2 (default is set to 1.5) 
# followed by allowable % error (default is set to 1%)

temp_dist_np_array,error=analyse_plate(5,5,75,100,50,0,1.5,1)
print(temp_dist_np_array)
print(error)

Enter the default boundary temp of the rectangular plate25
Enter the element size on X-axis1
Enter the element size on Y-axis1
[[ 87.5        100.         100.         100.          75.        ]
 [ 75.          78.6205368   76.07575113  69.64983929  50.        ]
 [ 75.          63.13778968  56.12793419  52.45168955  50.        ]
 [ 75.          42.83795818  33.21069728  33.99010903  50.        ]
 [ 37.5          0.           0.           0.          25.        ]]
0.6576530356008721


In [9]:
# example 2 custom analysis
# create a plate 5 X 5 (which means array that represents mesh points)
# set the boundary condition for the array (temperature data)
# pass the initalized array in to libman_elip_pde_solver
# solution array along with error will be returned

def_boundary_temp=25
plate_1=create_plate(5,5)
initalized_plate_1= boundary_conditions(plate_1, bottom=0,top=100,right=50,left=75)
array,error=libman_elip_pde_solver(initalized_plate_1,lamb=1.5,error=1)
print(array)
print(error)

Enter the element size on X-axis1
Enter the element size on Y-axis1
[[ 87.5        100.         100.         100.          75.        ]
 [ 75.          78.6205368   76.07575113  69.64983929  50.        ]
 [ 75.          63.13778968  56.12793419  52.45168955  50.        ]
 [ 75.          42.83795818  33.21069728  33.99010903  50.        ]
 [ 37.5          0.           0.           0.          25.        ]]
0.6576530356008721
